In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import sklearn.manifold as nonlin
import copy
import pickle as pk


In [2]:
print(torch.cuda.is_available())

True


## Datasets


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4
#cifar10
trainset_cifar = torchvision.datasets.CIFAR10(root='../data/', train=True,
                                        download=True, transform=transform)
trainset_cifar, validset_cifar = torch.utils.data.random_split(trainset_cifar, [45000,5000])



trainloader_cifar = torch.utils.data.DataLoader(trainset_cifar, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader_cifar = torch.utils.data.DataLoader(validset_cifar, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset_cifar = torchvision.datasets.CIFAR10(root='../data/', train=False,
                                       download=True, transform=transform)
testloader_cifar = torch.utils.data.DataLoader(testset_cifar, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes_cifar = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# MNIST DATASET
mnist_train = torchvision.datasets.MNIST(root='../data/', train=True,
                                         download=True, transform=transform)
mnist_train, mnist_valid = torch.utils.data.random_split(mnist_train, [50000,10000])
mnist_test = torchvision.datasets.MNIST(root='../data/', train=False,
                                        download=True, transform=transform)

trainloader_mnist = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                                shuffle=True, num_workers=2)
validloader_mnist =  torch.utils.data.DataLoader(mnist_valid, batch_size=batch_size,
                                                shuffle=True, num_workers=2)                                          
testloader_mnist = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
classes_mnist =  ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')


Files already downloaded and verified
Files already downloaded and verified


# functinons

In [3]:
def are_weights(model,weights):
    w1,w2,w3 = weights
    model.classifier[0].weight.data = w1
    model.classifier[3].weight.data = w2
    model.classifier[6].weight.data = w3
    return model


def construct_model(model, layers):
    l1, l2, l3 = layers
    model.classifier[0] = l1
    model.classifier[3] = l2
    model.classifier[6] = l3
    return model
    
def change_dimensionality(weight, dr_method):
    x1 = weight.detach().cpu().numpy().T
    x1 = dr_method.fit_transform(x1)
    x1 = torch.tensor(x1.T, dtype=torch.float32).to(device)
    print("x_new vector")
    print(x1.shape)
    return x1

    

In [4]:
def drprojection(model, method, whiten):
    weight_list = []
    weight1 = model.classifier[0].weight
    print('fc1 layer weight')

    fc1_reduced = change_dimensionality(
        weight1, method(n_components=int(weight1.shape[0]/2),  whiten=whiten, random_state=0, tol= 10))
    weight_list.append(fc1_reduced)
    x2 = torch.mm(model.classifier[0].weight, fc1_reduced.T)
    print(x2.shape)
    weight2 = torch.mm(model.classifier[3].weight, x2)
    print(weight2.shape)

    fc2_reduced = change_dimensionality(
        weight2, method(n_components=int(weight2.shape[0]/2),whiten=whiten, random_state=0, tol= 10))
    print('fc2 layer weight')
    print(fc2_reduced.shape)
    weight_list.append(fc2_reduced)

    x3_reduced = torch.mm(weight2, fc2_reduced.T)

    fc3_reduced = torch.mm(model.classifier[6].weight, x3_reduced)
    print('fc3 layer weight')
    print(fc3_reduced.shape)
    weight_list.append(fc3_reduced)

    return weight_list


In [8]:
device = torch.device('cuda:0')

In [5]:
def average_accuracy(net, dataset):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in dataset:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

def every_class_accuracy(model, testloader, classes):
    cor_pred = {classname: 0 for classname in classes}
    t_pred = {classname: 0 for classname in classes}
    with torch.no_grad():
        for data in testloader:
            im, labels = data
            output = model(im)
            _, predictions = torch.max(output, dim=1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    cor_pred[classes[label]] += 1
                t_pred[classes[label]] += 1
    
    for classname, correct_count in cor_pred.items():
        accuracy = 100 * float(correct_count) / t_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

        

In [6]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim


def training(model, trainset,valset, n, filename):

    running_loss = 0.0
    total = 0.0
    correct = 0.0
    loss_nodr = []
    acc_nodr = []
    
    val_loss_nodr = []
    model = model.to(device)
    val_acc_nodr = []
  

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    print(optimizer)

    for epoch in range(n):

        print('epoch:', epoch)
        for i, data in enumerate(trainset):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(True)
            with torch.set_grad_enabled(True):
                with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
                 
                 outputs = model(inputs).to(device)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                running_loss += loss.item()
        model.eval()
        val_loss = 0.0
        total_val=0.0
        correct_val = 0.0

        for i, data in enumerate(valset):

            inputs, labels2 = data
            inputs = inputs.to(device)
            labels2 = labels2.to(device)
            val_output  = model(inputs).to(device)
            loss = criterion(val_output, labels2)
            val_loss+=loss.item()
            total_val+=labels2.size(0)
            _, pred = torch.max(val_output.data, 1)
            correct_val += (pred == labels2).sum().item()

            
            
        train_loss = running_loss/len(trainset)
        acc_temp = 100 * correct / total

        valid_loss_temp = val_loss/len(valset)
        valid_acc_temp = (100 * correct_val)/total_val
        loss_nodr.append(train_loss)
        acc_nodr.append(acc_temp)

        val_loss_nodr.append(valid_loss_temp)
        val_acc_nodr.append(valid_acc_temp)
        print(
            f'[{epoch + 1}, {i + 1:5d}] train loss: {train_loss:.3f} train acc: {acc_temp:.3f}', 
            f'valid acc: {valid_acc_temp:.3f} valid loss  {valid_loss_temp:.3f} ')
        
        running_loss = 0.0
        correct = 0.0
        total = 0.0

    print('Finished Training')
    pickle_accloss(acc_nodr, loss_nodr,val_acc_nodr,val_loss_nodr,  filename)

    
def pickle_accloss(acc, loss,valid_acc, valid_loss, filename):
    accandloss = {'accuracy' : acc, 'loss' : loss, 'valid acc' : valid_acc, 'valid loss': valid_loss}
    with open('./data_vis/ICA/{}.pickle'.format(filename), 'wb') as ica_acc_and_loss:
        pk.dump(accandloss, ica_acc_and_loss)
    print('Saved dictionary of loss and accuracy!')



In [9]:
from torchvision.models import vgg16, VGG16_Weights
vggmodel = vgg16(weights=VGG16_Weights)
vggmodel = vggmodel.to(device)

In [14]:
vggmodel.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
training(vggmodel, trainloader_cifar, validloader_cifar, 1, 'vgg_cifar10_retrain')

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  1250] train loss: 1.075 train acc: 65.044 valid acc: 78.920 valid loss  0.634 
Finished Training


FileNotFoundError: [Errno 2] No such file or directory: './data_vis/ICA/vgg_cifar10_retrain.pickle'

In [13]:
average_accuracy(vggmodel, testloader_cifar)

Accuracy of the network on the 10000 test images: 0 %


In [22]:
for a,b in vggmodel.named_modules():
    if isinstance(b, nn.Linear):
        print(a)
        print(b)

classifier.0
Linear(in_features=25088, out_features=4096, bias=True)
classifier.3
Linear(in_features=4096, out_features=4096, bias=True)
classifier.6
Linear(in_features=4096, out_features=1000, bias=True)


In [21]:
def return_conv_layer_weights(model):
    vgg_weight_list = []
    for feature in model.features:
        if isinstance(feature, nn.Conv2d):
            print(feature.weight.shape)
            vgg_weight_list.append(feature.weight)
    return vgg_weight_list



def conv_reduction(matrix):
    matrix = matrix.reshape([matrix.shape[0]*3, matrix.shape[1]*3])
    matrix = matrix.T
    print(matrix.shape)
    new_dec = dec.FastICA(n_components=int(matrix.shape[1]/2), whiten='unit-variance', random_state=0)
    reduced_matrix = new_dec.fit_transform(matrix.detach().cpu().numpy())
    print(reduced_matrix.shape)
    reduced_matrix = reduced_matrix.T
    reduced_matrix = reduced_matrix.reshape((matrix.shape[0],matrix.shape[1],1,1))
    reduced_matrix = reduced_matrix.reshape([int(matrix.shape[0]/3),int(matrix.shape[1]/3),3,-1])
    print(reduced_matrix.shape)
    return torch.tensor(reduced_matrix, dtype=torch.float32).to(device)

def reduced_conv_layer_weights(conv_weight_matrices):
    reduced_vgg_params = []
    for w in conv_weight_matrices:
        r_w = conv_reduction(w)
        reduced_vgg_params.append(r_w)
    return reduced_vgg_params

    

In [22]:
weight_list_vgg = return_conv_layer_weights(vggmodel)
reduced_vgg_conv_weight_list = reduced_conv_layer_weights(weight_list_vgg)

torch.Size([64, 3, 3, 3])
torch.Size([64, 64, 3, 3])
torch.Size([128, 64, 3, 3])
torch.Size([128, 128, 3, 3])
torch.Size([256, 128, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([256, 256, 3, 3])
torch.Size([512, 256, 3, 3])
torch.Size([512, 512, 3, 3])
torch.Size([512, 512, 3, 3])
torch.Size([512, 512, 3, 3])
torch.Size([512, 512, 3, 3])
torch.Size([512, 512, 3, 3])
torch.Size([9, 192])


c:\Users\sulta\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\decomposition\_fastica.py:550: UserWarning: n_components is too large: it will be set to 9
  warnings.warn(


(9, 9)


c:\Users\sulta\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\decomposition\_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


ValueError: cannot reshape array of size 81 into shape (9,192,1,1)

In [38]:
ica_vggmodel = copy.deepcopy(vggmodel)

weight_list = drprojection(vggmodel,dec.FastICA, '‘unit-variance')


vgg_layers = (nn.Linear(25088, 2048, False), nn.Linear(
    2048, 2048, False), nn.Linear(2048, 1000, False))

ica_vggmodel = construct_model(
    ica_vggmodel, vgg_layers)
    
ica_vggmodel = are_weights(ica_vggmodel, tuple(weight_list))
ica_vggmodel.eval()


fc1 layer weight
x_new vector
torch.Size([2048, 25088])
torch.Size([4096, 2048])
torch.Size([4096, 2048])
x_new vector
torch.Size([2048, 2048])
fc2 layer weight
torch.Size([2048, 2048])
fc3 layer weight
torch.Size([1000, 2048])


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [39]:
average_accuracy(ica_vggmodel, testloader_cifar)

Accuracy of the network on the 10000 test images: 77 %
